In [3]:
# Given users review predit overall rating given by user
import json
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
        


In [4]:
# read json data
raw_data = pd.read_json("./dataset/Music_Review_train.json",lines=True)
X=raw_data['reviewText']
Y=raw_data['overall']
# preprocess data
def preprocess_data(X):
    sw_nltk=set(stopwords.words('english'))
    
    output=[]
    for sentence in X:
        # tokenize sentence
        tokens=word_tokenize(sentence)
        # normalize
        tokens=[w.lower() for w in tokens]
        # remove punctuation
        table=str.maketrans('','',string.punctuation)
        stripped=[w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        words=[word for word in stripped if word.isalpha()]
        # filter out stop words
        words=[w for w in words if not w in sw_nltk]
        # filter out stem words
        ps=PorterStemmer()
        words=[ps.stem(w) for w in words]
        output.append(words)
    return output
temp_X=preprocess_data(X)


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AKFE1P1ZDBPXU,B000002IWQ,"M. B. Link ""wildfirelink""","[4, 5]",Counterparts(1993). Rush's fifteenth studio al...,4,Underrated and Overrated Simultaneously,1067904000,"11 4, 2003"
1,A10CBLMJD1OQL7,B000002IWQ,mcmahonryan@hotmail.com,"[0, 0]","You have to ask yourself why, in 1998, there a...",4,Rush continues to reinvent itself,900633600,"07 17, 1998"
2,A331OF34UX4Y1K,B000002IWQ,"M. Fonseca ""carmarthen""","[2, 9]",I've bought this CD a long time ago and until ...,2,THE WEAKEST RUSH ALBUM EVER !,944006400,"12 1, 1999"
3,A18IZ5QX2PRB3C,B000002IWQ,Michael Stack,"[80, 83]","I am often in the minority in my view of this,...",5,Rush's '90s masterpiece.,1111536000,"03 23, 2005"
4,A2H22I2RWE0HOV,B000002IWQ,Mr. Sinister,"[3, 6]",The first thing that becomes apparent as soon ...,4,Counterparts Shifts The Rush Sound Once Again,1184025600,"07 10, 2007"


0    4
1    4
2    2
3    5
4    4
Name: overall, dtype: int64

In [ ]:
def get_vocabulary(X_c):
    class_vocab=[]
    unique,counts=np.unique(X_c,return_counts=True)
    vocab=dict(zip(unique,counts))
    class_vocab.append(vocab)    
    for class_label in range(5):
        unique,counts=np.unique(X_c[np.where(Y==class_label)],return_counts=True)
        vocab=dict(zip(unique,counts))
        class_vocab.append(vocab)
    return class_vocab
    
    # # vocab is list of dictionaries
    
    # vocab=[]
    # # map to store frequency
    # maps=[{},{},{},{},{}]
    # class_occur=[0,0,0,0,0]
    # for i in range(len(X_c)):
    #     # iterate over ith example
    #     class_label=Y[i]-1
    #     class_occur[class_label]+=1
    #     for word in X_c[i]:
    #         # iterate over features of ith example
    #         if word not in vocab[class_label]:
    #             vocab[class_label].add(word)
    #         else:
    #             maps[class_label][word]+=1
    # return vocab,map

# Naive bayes from scratch
def naive_bayes(X,Y,vocab):
    # number of examples = m
    m=len(X)
    # class then vocabulary
    psi_i=[[]]
    psi_y=None
    # get vocabulary
    vocab=get_vocabulary(X)
    iter=0
    for class_label in range(5):
        denom=sum([len(row) for row in X[np.where(Y==class_label)]])
        for word in vocab[0]:
            psi_i[class_label].append(vocab[class_label+1][word]/denom)
            iter+=1
    psi_y=len(np.where(Y==1))/m
    return psi_i,psi_y

def predict(x,psi_i,psi_y):
    class_prob=[]
    denom=1
    for class_label in range(5):
        numer=1
        for word in x:
            # start from here again
            numer*=psi_i[class_label][word]
            denom=None
        p_y_x=numer
        class_prob.append(p_y_x)
    class_prob/=denom
    return class_prob        

In [26]:
print(len(X))

50000


In [32]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91930\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [20]:
X=np.array([[1,2,3,4,5],[5,2,3,4,5],[1,2,3,4],[3,4,5],[1,2,3,4,5]])
Y=np.array([1,1,1,0,1])
len(X[np.where(Y==1)])
temp=sum([len(row) for row in X[np.where(Y==1)]]) 
print(temp)

19


<ipython-input-20-95fd7c8df35e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X=np.array([[1,2,3,4,5],[5,2,3,4,5],[1,2,3,4],[3,4,5],[1,2,3,4,5]])


In [16]:
print(np.where(Y==1))

(array([], dtype=int64),)
